In [1]:
import pandas as pd
import sys
import os
from pathlib import Path
import sqlite3
import warnings
warnings.filterwarnings("ignore")

project_root = os.path.abspath('../../')
if project_root not in sys.path:
    sys.path.append(project_root)
    
from src.data.utils import DBConnection
from src.config import DBConfig

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
class NotebookDBConnection(DBConnection):
    def __init__(self):
        db_path = os.path.join(project_root, 'data', 'pitcher_stats.db')
        super().__init__(db_name=db_path)

In [23]:
with NotebookDBConnection() as conn:
    df_team_mapping = pd.read_sql_query('''
            SELECT DISTINCT *
            FROM prediction_features
            WHERE game_date = '2025-04-15'
    ''', conn)


df_team_mapping

,pitcher_id,player_name,game_pk,game_date,home_team,away_team,team,opponent_team,is_home,p_throws,home_plate_umpire_x,k_per_9,ewma_3g_strikeouts,ewma_5g_strikeouts,ewma_10g_strikeouts,ewma_3g_batters_faced,ewma_5g_batters_faced,ewma_10g_batters_faced,ewma_3g_innings_pitched,ewma_5g_innings_pitched,ewma_10g_innings_pitched,ewma_3g_total_pitches,ewma_5g_total_pitches,ewma_10g_total_pitches,ewma_3g_avg_velocity,ewma_5g_avg_velocity,ewma_10g_avg_velocity,ewma_3g_max_velocity,ewma_5g_max_velocity,ewma_10g_max_velocity,ewma_3g_zone_percent,ewma_5g_zone_percent,ewma_10g_zone_percent,ewma_3g_swinging_strike_percent,ewma_5g_swinging_strike_percent,ewma_10g_swinging_strike_percent,ewma_3g_fastball_percent,ewma_5g_fastball_percent,ewma_10g_fastball_percent,ewma_3g_breaking_percent,ewma_5g_breaking_percent,ewma_10g_breaking_percent,ewma_3g_offspeed_percent,ewma_5g_offspeed_percent,ewma_10g_offspeed_percent,ewma_3g_k_percent,ewma_5g_k_percent,ewma_10g_k_percent,ewma_3g_k_per_9,ewma_5g_k_per_9,ewma_10g_k_per_9,strikeouts_lag1,strikeouts_lag2,strikeouts_change,strikeouts_volatility_3g,strikeouts_volatility_5g,strikeouts_volatility_10g,strikeouts_last2g_vs_baseline,innings_pitched_lag1,innings_pitched_lag2,innings_pitched_change,innings_pitched_volatility_3g,innings_pitched_volatility_5g,innings_pitched_volatility_10g,innings_pitched_last2g_vs_baseline,batters_faced_lag1,batters_faced_lag2,batters_faced_change,batters_faced_volatility_3g,batters_faced_volatility_5g,batters_faced_volatility_10g,batters_faced_last2g_vs_baseline,swinging_strike_percent_lag1,swinging_strike_percent_lag2,swinging_strike_percent_change,swinging_strike_percent_volatility_3g,swinging_strike_percent_volatility_5g,swinging_strike_percent_volatility_10g,swinging_strike_percent_last2g_vs_baseline,avg_velocity_lag1,avg_velocity_lag2,avg_velocity_change,avg_velocity_volatility_3g,avg_velocity_volatility_5g,avg_velocity_volatility_10g,avg_velocity_last2g_vs_baseline,k_percent_lag1,k_percent_lag2,k_percent_change,k_percent_volatility_3g,k_percent_volatility_5g,k_percent_volatility_10g,k_percent_last2g_vs_baseline,k_per_9_lag1,k_per_9_lag2,k_per_9_change,k_per_9_volatility_3g,k_per_9_volatility_5g,k_per_9_volatility_10g,k_per_9_last2g_vs_baseline,k_trend_up_lagged,k_trend_down_lagged,days_since_last_game,rest_days_4_less,rest_days_5,rest_days_6_more,fastball_percent_vs_baseline,breaking_percent_vs_baseline,offspeed_percent_vs_baseline,velocity_vs_baseline,significant_velo_drop,opp_k_percent,opp_bb_percent,opp_swing_percent,opp_contact_percent,opp_swinging_strike_percent,opp_chase_percent,opp_zone_contact_percent,home_plate_umpire_y,umpire_historical_k_per_9,pitcher_umpire_k_boost,ewma_5g_k_percent_vs_lhb_x,ewma_5g_k_percent_vs_rhb_x,ewma_5g_k_percent_vs_lhb_y,ewma_5g_k_percent_vs_rhb_y,ewma_5g_k_percent_vs_lhb,ewma_5g_k_percent_vs_rhb,p_throws_encoded,opponent_team_encoded,home_team_encoded,home_plate_umpire_encoded


In [22]:
with NotebookDBConnection() as conn:
    df = pd.read_sql_query('''
            SELECT DISTINCT *
            FROM game_level_team_stats
            WHERE game_date = '2025-04-15' 
    ''', conn)

df

,game_pk,game_date,team,opponent,home_team,away_team,is_home,pa,strikeouts,walks,hits,pitches_faced,swings,contact,swinging_strikes,zone_pitches,chases,zone_swings,zone_contact,season,k_percent,bb_percent,swing_percent,contact_percent,swinging_strike_percent,chase_percent,zone_contact_percent
0,778302,2025-04-15,COL,LAD,LAD,COL,0,36,40,20,20,604,324,228,96,288,316,204,160,2025,1.111111,0.555556,0.536424,0.703704,0.158940,0.379747,0.784314
1,778302,2025-04-15,LAD,COL,LAD,COL,1,43,20,32,40,680,300,224,76,340,340,216,176,2025,0.465116,0.744186,0.441176,0.746667,0.111765,0.247059,0.814815
2,778306,2025-04-15,LAA,TEX,TEX,LAA,0,33,44,8,20,544,288,200,88,310,234,232,184,2025,1.333333,0.242424,0.529412,0.694444,0.161765,0.239316,0.793103
3,778306,2025-04-15,TEX,LAA,TEX,LAA,1,33,32,8,32,504,248,196,52,275,229,192,164,2025,0.969697,0.242424,0.492063,0.790323,0.103175,0.244541,0.854167
4,778307,2025-04-15,MIN,NYM,MIN,NYM,1,37,20,0,52,604,292,256,36,306,298,204,192,2025,0.540541,0.000000,0.483444,0.876712,0.059603,0.295302,0.941176
5,778307,2025-04-15,NYM,MIN,MIN,NYM,0,37,28,4,28,480,272,208,64,296,184,216,176,2025,0.756757,0.108108,0.566667,0.764706,0.133333,0.304348,0.814815
6,778308,2025-04-15,HOU,STL,STL,HOU,0,37,32,12,28,544,236,212,24,261,283,180,168,2025,0.864865,0.324324,0.433824,0.898305,0.044118,0.197880,0.933333
7,778308,2025-04-15,STL,HOU,STL,HOU,1,33,20,8,24,564,220,160,60,283,281,156,124,2025,0.606061,0.242424,0.390071,0.727273,0.106383,0.227758,0.794872
8,778310,2025-04-15,DET,MIL,MIL,DET,0,31,40,12,4,540,224,168,56,302,238,176,144,2025,1.290323,0.387097,0.414815,0.750000,0.103704,0.201681,0.818182
9,778310,2025-04-15,MIL,DET,MIL,DET,1,39,20,20,36,700,296,236,60,350,350,236,196,2025,0.512821,0.512821,0.422857,0.797297,0.085714,0.171429,0.830508


In [26]:
with NotebookDBConnection() as conn:
    cursor = conn.cursor()
    
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print([table[0] for table in tables])

['statcast_pitchers', 'statcast_batters', 'pitcher_mapping', 'historical_umpire_data', 'team_mapping', 'mlb_api', 'game_level_pitchers', 'game_level_team_stats', 'historical_features_basic_train', 'historical_features_basic_test']


In [25]:
with NotebookDBConnection() as conn:
    cursor = conn.cursor()
    
    cursor.execute("DROP TABLE IF EXISTS prediction_evaluation")

In [27]:
with NotebookDBConnection() as conn:
    cursor = conn.cursor()
    cursor.execute("PRAGMA table_info(statcast_pitchers)")
    columns = cursor.fetchall()
    print([column[1] for column in columns])

['pitch_type', 'game_date', 'release_speed', 'release_pos_x', 'release_pos_z', 'player_name', 'batter', 'pitcher', 'events', 'description', 'spin_dir', 'spin_rate_deprecated', 'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des', 'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type', 'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y', 'tfs_deprecated', 'tfs_zulu_deprecated', 'umpire', 'sv_id', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot', 'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension', 'game_pk', 'fielder_2', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y', 'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle', 'woba_value', 'woba_denom', 'babip_value', 'iso_value', 'launch_speed_a

In [ ]:
with NotebookDBConnection() as conn:
    df = pd.read_sql_query("SELECT * FROM game_level_pitchers ORDER BY RANDOM() LIMIT 10", conn)
df


,pitcher_id,game_pk,game_date,player_name,home_team,away_team,p_throws,is_home,strikeouts,batters_faced,total_pitches,total_swinging_strikes,total_called_strikes,total_fastballs,total_breaking,total_offspeed,total_in_zone,avg_velocity,max_velocity,avg_spin_rate,avg_horizontal_break,avg_vertical_break,season,innings_pitched,k_percent,k_per_9,swinging_strike_percent,called_strike_percent,fastball_percent,breaking_percent,offspeed_percent,zone_percent,opponent_team
0,570632,449146,2016-09-22,"Ureña, José",MIA,ATL,R,1,6,9,96,6,16,44,22,30,40,90.606250,96.9,2053.288889,-0.719792,0.793542,2016,3.2,0.666667,16.875000,0.062500,0.166667,0.458333,0.229167,0.312500,0.416667,ATL
1,666200,746089,2024-03-28,"Luzardo, Jesús",MIA,PIT,L,1,8,9,85,17,12,42,27,16,35,90.941176,97.6,2213.894118,0.853647,0.775765,2024,5.2,0.888889,13.846154,0.200000,0.141176,0.494118,0.317647,0.188235,0.411765,PIT
2,676879,663178,2022-04-07,"Ashby, Aaron",CHC,MIL,L,0,2,8,82,8,10,32,34,16,42,88.319512,96.7,2193.780488,0.353415,0.216829,2022,2.2,0.250000,8.181818,0.097561,0.121951,0.390244,0.414634,0.195122,0.512195,CHC
3,506433,565199,2019-09-17,"Darvish, Yu",CHC,CIN,R,1,26,9,204,32,48,64,120,20,106,88.004902,96.9,2532.630952,0.329510,0.234118,2019,7.2,2.888889,32.500000,0.156863,0.235294,0.313725,0.588235,0.098039,0.519608,CIN
4,608566,633035,2021-08-06,"Márquez, Germán",COL,MIA,R,1,14,10,152,24,22,92,60,0,78,91.965789,97.9,2286.960526,-0.204868,0.523684,2021,6.2,1.400000,20.322581,0.157895,0.144737,0.605263,0.394737,0.000000,0.513158,MIA
5,596112,491621,2017-07-27,"Stephenson, Robert",MIA,CIN,R,0,4,9,184,14,26,96,56,30,74,89.230435,95.3,2205.847826,-0.578804,1.207283,2017,5.2,0.444444,6.923077,0.076087,0.141304,0.521739,0.304348,0.163043,0.402174,MIA
6,674003,744966,2024-09-05,"Bradford, Cody",TEX,LAA,L,1,8,9,180,12,38,104,40,36,94,84.122222,89.7,1965.988889,0.495667,0.995778,2024,6.2,0.888889,11.612903,0.066667,0.211111,0.577778,0.222222,0.200000,0.522222,LAA
7,518489,717506,2023-07-03,"Brasier, Ryan",LAD,PIT,R,1,2,6,21,2,4,19,2,0,11,91.552381,96.4,2380.047619,0.028095,0.811905,2023,1.2,0.333333,15.000000,0.095238,0.190476,0.904762,0.095238,0.000000,0.523810,PIT
8,571863,716469,2023-09-23,"Koch, Matt",CHC,COL,R,0,4,6,42,6,10,38,4,0,26,90.195238,94.1,2117.809524,-0.252381,0.864286,2023,1.2,0.666667,30.000000,0.142857,0.238095,0.904762,0.095238,0.000000,0.619048,CHC
9,664641,448547,2016-08-09,"Miranda, Aríel",SEA,DET,L,1,2,5,34,2,6,18,8,8,16,89.641176,96.4,2055.764706,1.041765,0.830588,2016,1.2,0.400000,15.000000,0.058824,0.176471,0.529412,0.235294,0.235294,0.470588,DET


: 